In [1]:
!nvidia-smi

Sun Dec  6 01:42:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### IMPORTS

In [3]:
import os, sys, math
import numpy as np
import torch
import torch.nn as nn
from collections import defaultdict
from torch.utils.data import Dataset
from skimage.transform import resize
import torchvision.transforms as transforms
from PIL import Image
from torch.autograd import Variable
from os import listdir
from os.path import isfile, join

### DATA - One time installation

In [4]:
!pip install openimages

     |████████████████████████████████| 133kB 5.1MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 71kB 4.7MB/s 
     |████████████████████████████████| 7.0MB 5.9MB/s 
     |████████████████████████████████| 296kB 52.5MB/s 
     |████████████████████████████████| 137.3MB 39kB/s 
ERROR: botocore 1.19.30 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [5]:
!oi_download_dataset --base_dir '/content/drive/My Drive/CVTermProject/train/data' --labels Ambulance Building Person --format darknet --limit 2000

tcmalloc: large alloc 1194041344 bytes == 0x5a26a000 @  0x7f9b9da821e7 0x59211c 0x598dce 0x50a1cc 0x50beb4 0x507be4 0x588c8b 0x4b6271 0x55dc5d 0x50bddb 0x507be4 0x508ec2 0x594a01 0x59fd0e 0x50d256 0x507be4 0x508ec2 0x594a01 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x509900 0x50a2fd 0x50cc96 0x5095c8 0x50a2fd 0x50beb4
2020-12-06  01:42:06 INFO NumExpr defaulting to 4 threads.
2020-12-06  01:42:08 INFO Downloading 207 train images for class 'ambulance'
 19% 40/207 [00:09<00:48,  3.45it/s]^C


In [6]:
# !oi_download_dataset --base_dir '/content/drive/My Drive/CVTermProject/train/data' --labels Ambulance --format darknet --limit 1000

In [7]:
!mkdir '/content/drive/My Drive/CVTermProject/train/data/dataset/'

mkdir: cannot create directory ‘/content/drive/My Drive/CVTermProject/train/data/dataset/’: File exists


In [8]:
!mkdir '/content/drive/My Drive/CVTermProject/train/data/dataset/labels'

mkdir: cannot create directory ‘/content/drive/My Drive/CVTermProject/train/data/dataset/labels’: File exists


In [9]:
!mkdir '/content/drive/My Drive/CVTermProject/train/data/dataset/images'

mkdir: cannot create directory ‘/content/drive/My Drive/CVTermProject/train/data/dataset/images’: File exists


In [ ]:
!cp -r '/content/drive/MyDrive/CVTermProject/train/data/person/darknet/.' '/content/drive/My Drive/CVTermProject/train/data/dataset/labels'

In [ ]:
!cp -r '/content/drive/MyDrive/CVTermProject/train/data/ambulance/darknet/.' '/content/drive/My Drive/CVTermProject/train/data/dataset/labels'

In [ ]:
!cp -r '/content/drive/MyDrive/CVTermProject/train/data/building/darknet/.' '/content/drive/My Drive/CVTermProject/train/data/dataset/labels'

In [ ]:
!cp -r '/content/drive/MyDrive/CVTermProject/train/data/building/images/.' '/content/drive/My Drive/CVTermProject/train/data/dataset/images'

In [ ]:
!cp -r '/content/drive/MyDrive/CVTermProject/train/data/person/images/.' '/content/drive/My Drive/CVTermProject/train/data/dataset/images'

In [ ]:
!cp -r '/content/drive/MyDrive/CVTermProject/train/data/ambulance/images/.' '/content/drive/My Drive/CVTermProject/train/data/dataset/images'

In [ ]:
!ls '/content/drive/My Drive/CVTermProject/train/data/dataset/images' | wc -l

### Load the data - One time installation

In [ ]:
import glob
import os, sys
import numpy as np


In [ ]:
files = []

In [ ]:
PATH = '/content/drive/My Drive/CVTermProject/train/data/dataset/images'
counter = 1

for pathAndFileName in glob.iglob(os.path.join(PATH , "*.jpg")):
  title, ext = os.path.splitext(os.path.basename(pathAndFileName))
  files.append(PATH + "/" + title + '.jpg')
  # if counter == index_test:
  #   counter = 1
  #   vali_file.append(PATH + "/" + title + '.jpg')
  # else:
  #   train_f.append(PATH + "/" + title + '.jpg')
  #   counter += 1

import random

random.shuffle(files)

In [ ]:
len(files)

In [ ]:
split_pct = int(len(files) * 0.9)
split_pct

In [ ]:
train_f = files[len(files) - split_pct:]
vali_file = files[:len(files) - split_pct]

In [ ]:
len(train_f)

In [ ]:
len(vali_file)

In [ ]:
with open("/content/drive/My Drive/CVTermProject/train/data/train.txt", "w") as train:
  for i in train_f:
    train.write(i + "\n")

with open("/content/drive/My Drive/CVTermProject/train/data/val.txt", "w") as validation:
  for i in vali_file:
    validation.write(i + "\n")


In [ ]:
!cat '/content/drive/My Drive/CVTermProject/train/data/val.txt' | wc -l

In [ ]:
!cat '/content/drive/My Drive/CVTermProject/train/data/train.txt' | wc -l

### CONFIGURATION

In [4]:
TRAIN_PATH = '/content/drive/My Drive/CVTermProject/train/data/train.txt'

In [ ]:
def get_tensor(img, img_shape):
  img_shape = (img_shape, img_shape)
  
  h, w, _ = img.shape
  dim_diff = np.abs(h - w)
  pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
  pad = ((pad1, pad2), (0, 0), (0, 0)) if h <= w else ((0, 0), (pad1, pad2), (0, 0))
  input_img = np.pad(img, pad, 'constant', constant_values=128) / 255.
  padded_h, padded_w, _ = input_img.shape
  input_img = resize(input_img, (*img_shape, 3), mode='reflect')
  input_img = np.transpose(input_img, (2, 0, 1))
  input_img = torch.from_numpy(input_img).float()
  return input_img, (h, w)

In [ ]:
def img_to_tensor():
  new_files = []
  count = 0
  with open(TRAIN_PATH, 'r') as file:
    for filename in file.readlines():
      filename = filename.rstrip()
      img = np.array(Image.open(filename))
      # print(filename)
      # print(img.shape)
      new_file = filename.replace('images', 'tensors').replace('.jpg', '.pt')
      if len(img.shape) == 3:
        torch.save(get_tensor(img, 416), new_file)
      count += 1
      if count%50 == 0:
        print('here')
        # break

In [ ]:
img_to_tensor()

In [6]:
!ls '/content/drive/MyDrive/CVTermProject/train/data/dataset/tensors' | wc -l

1978


In [7]:
CLASS_PATH = "/content/drive/My Drive/CVTermProject/train/config/coco.names"
CONFIG_PATH = "/content/drive/My Drive/CVTermProject/train/config/coco.data"
MODEL_CONFIG_PATH = '/content/drive/My Drive/CVTermProject/train/config/yolov3.cfg'

In [8]:
def read_classes(path):
  with open(path, "r") as file:
    return file.read().strip().split("\n")

classes = read_classes(CLASS_PATH)
classes

['ambulance', 'building', 'person']

In [9]:
def read_data(path):
  with open(path, "r") as file:
    data = file.readlines()

  config = dict()
  for d in data:
    d = d.strip()
    if len(d) < 1:
      continue
    key, value = d.split("=")
    key = key.strip()
    value = value.strip()
    config[key] = value
  
  return config

config = read_data(CONFIG_PATH)
config

{'backup': 'backup/',
 'classes': '3',
 'names': '/content/drive/MyDrive/CVTermProject/train/config/coco.names',
 'train': '/content/drive/MyDrive/CVTermProject/train/data_old/train.txt',
 'valid': '/content/drive/MyDrive/CVTermProject/train/data_old/val.txt'}

In [10]:
NUM_CLASSES = int(config['classes'])
NUM_CLASSES

3

### YOLO

In [11]:
def parse_model_config(path):
    file = open(path, 'r')
    lines = file.read().split('\n')
    lines = [x for x in lines if x and not x.startswith('#')]
    lines = [x.rstrip().lstrip() for x in lines] # get rid of fringe whitespaces
    module_defs = []
    for line in lines:
        if line.startswith('['): # This marks the start of a new block
            module_defs.append({})
            module_defs[-1]['type'] = line[1:-1].rstrip()
            if module_defs[-1]['type'] == 'convolutional':
                module_defs[-1]['batch_normalize'] = 0
        else:
            key, value = line.split("=")
            value = value.strip()
            module_defs[-1][key.rstrip()] = value.strip()

    return module_defs

In [12]:
def bbox_iou(box1, box2, x1y1x2y2=True):
    if not x1y1x2y2:
        b1_x1, b1_x2 = box1[:, 0] - box1[:, 2] / 2, box1[:, 0] + box1[:, 2] / 2
        b1_y1, b1_y2 = box1[:, 1] - box1[:, 3] / 2, box1[:, 1] + box1[:, 3] / 2
        b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
        b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2
    else:
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]

    inter_rect_x1 = torch.max(b1_x1, b2_x1)
    inter_rect_y1 = torch.max(b1_y1, b2_y1)
    inter_rect_x2 = torch.min(b1_x2, b2_x2)
    inter_rect_y2 = torch.min(b1_y2, b2_y2)
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * torch.clamp(
        inter_rect_y2 - inter_rect_y1 + 1, min=0
    )
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)

    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)

    return iou

In [13]:
def build_targets(
    pred_boxes, pred_conf, pred_cls, target, anchors, num_anchors, num_classes, grid_size, ignore_thres, img_dim
):
    nB = target.size(0)
    nA = num_anchors
    nC = num_classes
    nG = grid_size
    mask = torch.zeros(nB, nA, nG, nG)
    conf_mask = torch.ones(nB, nA, nG, nG)
    tx = torch.zeros(nB, nA, nG, nG)
    ty = torch.zeros(nB, nA, nG, nG)
    tw = torch.zeros(nB, nA, nG, nG)
    th = torch.zeros(nB, nA, nG, nG)
    tconf = torch.ByteTensor(nB, nA, nG, nG).fill_(0)
    tcls = torch.ByteTensor(nB, nA, nG, nG, nC).fill_(0)

    nGT = 0
    nCorrect = 0
    for b in range(nB):
        for t in range(target.shape[1]):
            if target[b, t].sum() == 0:
                continue
            nGT += 1
            gx = target[b, t, 1] * nG
            gy = target[b, t, 2] * nG
            gw = target[b, t, 3] * nG
            gh = target[b, t, 4] * nG
            gi = int(gx)
            gj = int(gy)
            gt_box = torch.FloatTensor(np.array([0, 0, gw, gh])).unsqueeze(0)
            anchor_shapes = torch.FloatTensor(np.concatenate((np.zeros((len(anchors), 2)), np.array(anchors)), 1))
            anch_ious = bbox_iou(gt_box, anchor_shapes)
            conf_mask[b, anch_ious > ignore_thres, gj, gi] = 0
            best_n = np.argmax(anch_ious)
            gt_box = torch.FloatTensor(np.array([gx, gy, gw, gh])).unsqueeze(0)
            pred_box = pred_boxes[b, best_n, gj, gi].unsqueeze(0)
            
            mask[b, best_n, gj, gi] = 1
            conf_mask[b, best_n, gj, gi] = 1
            
            tx[b, best_n, gj, gi] = gx - gi
            ty[b, best_n, gj, gi] = gy - gj

            tw[b, best_n, gj, gi] = math.log(gw / anchors[best_n][0] + 1e-16)
            th[b, best_n, gj, gi] = math.log(gh / anchors[best_n][1] + 1e-16)
            target_label = int(target[b, t, 0])
            tcls[b, best_n, gj, gi, target_label] = 1
            tconf[b, best_n, gj, gi] = 1

            iou = bbox_iou(gt_box, pred_box, x1y1x2y2=False)
            pred_label = torch.argmax(pred_cls[b, best_n, gj, gi])
            score = pred_conf[b, best_n, gj, gi]
            if iou > 0.5 and pred_label == target_label and score > 0.5:
                nCorrect += 1

    return nGT, nCorrect, mask, conf_mask, tx, ty, tw, th, tconf, tcls

In [14]:
class EmptyLayer(nn.Module):

  def __init__(self):
    super(EmptyLayer, self).__init__()

In [15]:
class YOLO(nn.Module):
  def __init__(self, anchors, num_classes, dimensions):
    super(YOLO, self).__init__()
    self.anchors = anchors
    self.num_anchors = len(anchors)
    self.num_classes = num_classes
    self.bbox = 5 + num_classes
    self.dimensions = dimensions
    self.ignore_thres = 0.5
    self.lambda_coord = 1

    self.mse_loss = nn.MSELoss()
    self.bce_loss = nn.BCELoss()
    self.ce_loss = nn.CrossEntropyLoss()
  

  def forward(self, x, targets=None):
    # print("X IS CUDA OR NOT", x.is_cuda)
    nB = x.size(0)
    grid_size = x.size(2)

    stride = self.dimensions / grid_size

    FloatTensor = torch.cuda.FloatTensor if x.is_cuda else torch.FloatTensor
    LongTensor = torch.cuda.LongTensor if x.is_cuda else torch.LongTensor
    ByteTensor = torch.cuda.ByteTensor if x.is_cuda else torch.ByteTensor

    prediction = x.view(nB, self.num_anchors, self.bbox, grid_size, grid_size).permute(0, 1, 3, 4, 2).contiguous()

    x = torch.sigmoid(prediction[..., 0]) 
    y = torch.sigmoid(prediction[..., 1]) 
    w = prediction[..., 2]  
    h = prediction[..., 3]  
    confidence = torch.sigmoid(prediction[..., 4])  
    clas = torch.sigmoid(prediction[..., 5:])  
    scaled_anchors = FloatTensor([(i / stride, j / stride) for i, j in self.anchors])

    predicted_boxes = FloatTensor(prediction[..., :4].shape)
    predicted_boxes[..., 0] = x.data + torch.arange(grid_size).repeat(grid_size, 1).view([1, 1, grid_size, grid_size]).type(FloatTensor)
    predicted_boxes[..., 1] = y.data + torch.arange(grid_size).repeat(grid_size, 1).t().view([1, 1, grid_size, grid_size]).type(FloatTensor)
    predicted_boxes[..., 2] = torch.exp(w.data) * scaled_anchors[:, 0:1].view((1, self.num_anchors, 1, 1))
    predicted_boxes[..., 3] = torch.exp(h.data) * scaled_anchors[:, 1:2].view((1, self.num_anchors, 1, 1))

    if targets is not None:
      if x.is_cuda:
        self.mse_loss = self.mse_loss.cuda()
        self.bce_loss = self.bce_loss.cuda()
        self.ce_loss = self.ce_loss.cuda()

      nGT, nCorrect, mask, conf_mask, tx, ty, tw, th, tconf, tcls = build_targets(
          pred_boxes = predicted_boxes.cpu().data,
          pred_conf = confidence.cpu().data,
          pred_cls = clas.cpu().data,
          target=targets.cpu().data,
          anchors=scaled_anchors.cpu().data,
          num_anchors = self.num_anchors,
          num_classes = self.num_classes,
          grid_size=grid_size,
          ignore_thres=self.ignore_thres,
          img_dim = self.dimensions
      )

      proposals = int((confidence > 0.5).sum().item())
      recall = float(nCorrect / nGT) if nGT else 1

      precision = 0

      if proposals > 0:
        precision = float(nCorrect / proposals)

      mask = Variable(mask.type(ByteTensor))
      conf_mask = Variable(conf_mask.type(ByteTensor))

      tx = Variable(tx.type(FloatTensor), requires_grad=False)
      ty = Variable(ty.type(FloatTensor), requires_grad=False)
      tw = Variable(tw.type(FloatTensor), requires_grad=False)
      th = Variable(th.type(FloatTensor), requires_grad=False)
      tconf = Variable(tconf.type(FloatTensor), requires_grad=False)
      tcls = Variable(tcls.type(FloatTensor), requires_grad=False)

      conf_mask_true = mask
      conf_mask_false = conf_mask - mask

      loss_x = self.mse_loss(x[mask], tx[mask])
      loss_y = self.mse_loss(y[mask], ty[mask])
      loss_w = self.mse_loss(w[mask], tw[mask])
      loss_h = self.mse_loss(h[mask], th[mask])


      loss_conf = self.bce_loss(confidence[conf_mask_false], tconf[conf_mask_false]) + self.bce_loss(
                confidence[conf_mask_true], tconf[conf_mask_true])
      
      loss_cls = (1 / nB) * self.ce_loss(clas[mask], torch.argmax(tcls[mask], 1))
      loss = loss_x + loss_y + loss_w + loss_h + loss_conf + loss_cls

      return (loss, loss_x.item(), loss_y.item(), loss_w.item(), loss_h.item(), loss_conf.item(), loss_cls.item(), recall, precision)
    else:
      output = torch.cat((predicted_boxes.view(nB, -1, 4) * stride,
                         confidence.view(nB, -1, 1),
                         clas.view(nB, -1, self.num_classes)),
                         -1)
      
      return output

  

In [16]:
class Darknet(nn.Module):

  def __init__(self, config_path, img_size=416):
    super(Darknet, self).__init__()
    self.modules = parse_model_config(config_path)
    self.hyperparameters, self.pipeline = create_pipeline(self.modules)
    self.img_size = img_size
    self.seen = 0
    self.header_info = np.array([0, 0, self.seen, 0])
    self.loss_names = ["x", "y", "w", "h", "conf", "cls", "recall", "precision"]


  def forward(self, x, targets=None):
    is_training = targets is not None

    output = []
    self.losses = defaultdict(float)
    layer_output = []

    for i , (module_def, module) in enumerate(zip(self.modules, self.pipeline)):
      if module_def["type"] in ["convolutional", "upsample", "maxpool"]:
          x = module(x)
      elif module_def["type"] == "route":
          layer_i = [int(x) for x in module_def["layers"].split(",")]
          x = torch.cat([layer_output[i] for i in layer_i], 1)
      elif module_def["type"] == "shortcut":
          layer_i = int(module_def["from"])
          x = layer_output[-1] + layer_output[layer_i]
      elif module_def["type"] == "yolo":
          if is_training:
              x, *losses = module[0](x, targets)
              for name, loss in zip(self.loss_names, losses):
                  self.losses[name] += loss
          else:
              x = module(x)
          output.append(x)
      layer_output.append(x)


    self.losses["recall"] /= 3
    self.losses["precision"] /= 3
    return sum(output) if is_training else torch.cat(output, 1)

  def load_weights(self, weights_path):

    fp = open(weights_path, "rb")
    header = np.fromfile(fp, dtype=np.int32, count=5)  

    
    self.header_info = header

    self.seen = header[3]
    weights = np.fromfile(fp, dtype=np.float32)  
    fp.close()

    ptr = 0
    for i, (module_def, module) in enumerate(zip(self.modules, self.pipeline)):
      if module_def["type"] == "convolutional":
        conv_layer = module[0]
        if module_def["batch_normalize"]:
            bn_layer = module[1]
            num_b = bn_layer.bias.numel()  
            bn_b = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(bn_layer.bias)
            bn_layer.bias.data.copy_(bn_b)
            ptr += num_b
            bn_w = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(bn_layer.weight)
            bn_layer.weight.data.copy_(bn_w)
            ptr += num_b
            bn_rm = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(bn_layer.running_mean)
            bn_layer.running_mean.data.copy_(bn_rm)
            ptr += num_b
            bn_rv = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(bn_layer.running_var)
            bn_layer.running_var.data.copy_(bn_rv)
            ptr += num_b
        else:
            num_b = conv_layer.bias.numel()
            conv_b = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(conv_layer.bias)
            conv_layer.bias.data.copy_(conv_b)
            ptr += num_b
        num_w = conv_layer.weight.numel()
        conv_w = torch.from_numpy(weights[ptr : ptr + num_w]).view_as(conv_layer.weight)
        conv_layer.weight.data.copy_(conv_w)
        ptr += num_w


  def save_weights(self, path, cutoff=-1):

    fp = open(path, "wb")
    self.header_info[3] = self.seen
    self.header_info.tofile(fp)

    # Iterate through layers
    for i, (module_def, module) in enumerate(zip(self.modules[:cutoff], self.pipeline[:cutoff])):
        if module_def["type"] == "convolutional":
            conv_layer = module[0]
            # If batch norm, load bn first
            if module_def["batch_normalize"]:
                bn_layer = module[1]
                bn_layer.bias.data.cpu().numpy().tofile(fp)
                bn_layer.weight.data.cpu().numpy().tofile(fp)
                bn_layer.running_mean.data.cpu().numpy().tofile(fp)
                bn_layer.running_var.data.cpu().numpy().tofile(fp)
            # Load conv bias
            else:
                conv_layer.bias.data.cpu().numpy().tofile(fp)
            # Load conv weights
            conv_layer.weight.data.cpu().numpy().tofile(fp)

    fp.close()



In [17]:
def create_pipeline(modules):
  hyperparameters = modules.pop(0)
  output_filters = [int(hyperparameters['channels'])]
  pipeline = nn.ModuleList()

  for i, module in enumerate(modules):
    mod = nn.Sequential()
    # print(module)
    module_type = module['type']

    if module_type == 'convolutional':
      batch_norm = int(module['batch_normalize'])
      filters = int(module['filters'])

      kernel_size = int(module['size'])

      padding = (kernel_size - 1) // 2 if int(module['pad']) else 0

      mod.add_module("conv_%d" % i, nn.Conv2d(in_channels = output_filters[-1],
                                              out_channels = filters,
                                              kernel_size = kernel_size,
                                              stride = int(module['stride']),
                                              padding = padding,
                                              bias = not batch_norm))
      
      if batch_norm:
        mod.add_module('batch_norm_%d' % i, nn.BatchNorm2d(filters))

      if module['activation'] == 'leaky':
        mod.add_module('leaky_%d' % i, nn.LeakyReLU(0.1))

    elif module_type == 'upsample':
      stride = int(module['stride'])
      mod.add_module('upsample_%d' % i, nn.Upsample(scale_factor=stride, 
                                                    mode='nearest'))
      
    elif module_type == 'maxpool':
      kernel_size = int(module['size'])
      stride = int(module['stride'])

      if kernel_size == 2 and stride == 1:
        padding = nn.ZeroPad2d((0, 1, 0, 1))
        mod.add_module("_debug_padding_%d" % i, padding)

      mod.add_module("Maxpool_%d" % i, nn.MaxPool2d(kernel_size=kernel_size,
                                                    stride = stride,
                                                    padding=int((kernel_size - 1) // 2)))

    elif module_type == 'route':
      layers = [int(x) for x in module["layers"].split(",")]
      filters = sum([output_filters[layer_i] for layer_i in layers])
      mod.add_module('route_%d' % i, EmptyLayer())

    elif module_type == 'shortcut':
      filters = output_filters[int(module['from'])]
      mod.add_module("shortcut_%d" % i, EmptyLayer())

    elif module_type == 'yolo':
      anchor_idx = [int(x) for x in module['mask'].split(',')]
      anchors = [int(x) for x in module['anchors'].split(',')]

      anchors = [(anchors[i], anchors[i + 1]) for i in range(0, len(anchors), 2)]
      anchors = [anchors[i] for i in anchor_idx]

      num_classes = int(module['classes'])
      img_height = int(hyperparameters['height'])

      mod.add_module('yolo_%d' % i, YOLO(anchors, num_classes, img_height))
    pipeline.append(mod)
    output_filters.append(filters)

  return hyperparameters, pipeline

In [18]:
CLASS_PATH = '/content/drive/My Drive/CVTermProject/train/config/coco.names'
DATA_CONFIG_PATH = '/content/drive/My Drive/CVTermProject/train/config/coco.data'
MODEL_CONFIG_PATH = '/content/drive/My Drive/CVTermProject/train/config/yolov3.cfg'
WEIGHTS_PATH = '/content/drive/My Drive/CVTermProject/train/config/yolov3.weights'
BATCH_SIZE = 16
TRAIN_PATH = '/content/drive/MyDrive/CVTermProject/train/data/train.txt'
EPOCHS = 500
TENSOR_DATA = '/content/drive/MyDrive/CVTermProject/train/data/dataset/tensors'

In [19]:
class ListDataset(Dataset):
    def __init__(self, tensor_path, img_size=416):
        self.tensor_files = [f for f in listdir(tensor_path)]
        self.dir = tensor_path
        self.max_objects = 3

    def __getitem__(self, index):

        dir_path = join(self.dir, self.tensor_files[index])
        data_in = torch.load(dir_path)
        input_img = data_in[0]
        h = data_in[1][0]
        w = data_in[1][1]
        
        dim_diff = np.abs(h - w)
        pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
        pad = ((pad1, pad2), (0, 0), (0, 0)) if h <= w else ((0, 0), (pad1, pad2), (0, 0))
        max_pad = max(h, w)
        label_path = dir_path.replace('tensors', 'labels').replace('.pt', '.txt')
        # print(label_path)

        labels = None
        if os.path.exists(label_path):
            labels = np.loadtxt(label_path).reshape(-1, 5)
            x1 = w * labels[:, 1]
            y1 = h * labels[:, 2]
            x2 = w * labels[:, 3]
            y2 = h * labels[:, 4]

            x1 += pad[1][0]
            y1 += pad[0][1]

            labels[:, 1] = x1 / max_pad
            labels[:, 2] = y1 / max_pad
            labels[:, 3] = x2 / max_pad
            labels[:, 4] = y2 / max_pad
            
        max_objects = self.max_objects
        filled_labels = np.zeros((max_objects, 5))
        if labels is not None:
            filled_labels[range(len(labels))[:max_objects]] = labels[:max_objects]
        filled_labels = torch.from_numpy(filled_labels)

        return input_img, filled_labels

    def __len__(self):
        return len(self.tensor_files)

In [20]:
d = ListDataset(TENSOR_DATA)[1]
d

(tensor([[[0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          ...,
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020]],
 
         [[0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          ...,
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020]],
 
         [[0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ..., 0.5020, 0.5020, 0.5020],
          [0.5020, 0.5020, 0.5020,  ...,

In [21]:
cuda = torch.cuda.is_available()

os.makedirs("/content/drive/My Drive/CVTermProject/train/checkpoints/new", exist_ok=True)
classes = read_classes(CLASS_PATH)
data_config = read_data(DATA_CONFIG_PATH)
train_path = data_config["train"]

train_path

'/content/drive/MyDrive/CVTermProject/train/data_old/train.txt'

In [22]:
classes

['ambulance', 'building', 'person']

In [23]:
model = Darknet(MODEL_CONFIG_PATH)
model.load_weights(WEIGHTS_PATH)

if cuda:
  model = model.cuda()

model.train()
cuda

True

In [24]:
dataloader = torch.utils.data.DataLoader(
    ListDataset(TENSOR_DATA), batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True
)

In [25]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [26]:
optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, model.parameters()))

In [27]:
import time

In [ ]:
for epoch in range(EPOCHS):
  model_start = time.time()
  for batch_i, (imgs, targets) in enumerate(dataloader):
    imgs = Variable(imgs.type(Tensor))
    target = Variable(targets.type(Tensor), requires_grad=False)

    # print(imgs, target)
    
    start_time = time.time()

    optimizer.zero_grad()

    loss = model(imgs, targets)
    loss.backward()
    optimizer.step()
    end_time = time.time() - start_time

    print(
            "[Epoch %3d/%3d, Batch %3d/%3d] [Time Taken: %s][Losses: conf %f, cls %f, total %f, recall: %.5f, precision: %.5f]"
            % (
                epoch,
                EPOCHS,
                batch_i,
                len(dataloader),
                end_time,
                model.losses["conf"],
                model.losses["cls"],
                loss.item(),
                model.losses["recall"],
                model.losses["precision"],
               
            )
        )
    start_time = time.time()
    model.seen += imgs.size(0)
  
  # if epoch % 10 == 0:
  model.save_weights("%s/%d.weights" % ('/content/drive/My Drive/CVTermProject/train/checkpoints/new', epoch))
  # print("EPOCH TIME: ", time.time() - model_start)
  # model_start = time.time()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:96: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:98: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/pytho

Streaming output truncated to the last 5000 lines.
[Epoch 459/500, Batch  84/124] [Time Taken: 0.3985018730163574][Losses: conf 0.020161, cls 0.104412, total 0.199806, recall: 0.98611, precision: 0.34145]
[Epoch 459/500, Batch  85/124] [Time Taken: 0.3820781707763672][Losses: conf 0.018341, cls 0.103646, total 0.175322, recall: 1.00000, precision: 0.33989]
[Epoch 459/500, Batch  86/124] [Time Taken: 0.36702871322631836][Losses: conf 0.019501, cls 0.106148, total 0.184389, recall: 1.00000, precision: 0.26509]
[Epoch 459/500, Batch  87/124] [Time Taken: 0.4082026481628418][Losses: conf 0.016241, cls 0.103527, total 0.175579, recall: 1.00000, precision: 0.39131]
[Epoch 459/500, Batch  88/124] [Time Taken: 0.40377354621887207][Losses: conf 0.021414, cls 0.106254, total 0.176309, recall: 0.97619, precision: 0.36806]
[Epoch 459/500, Batch  89/124] [Time Taken: 0.4028618335723877][Losses: conf 0.015883, cls 0.106080, total 0.170760, recall: 0.97436, precision: 0.36989]
[Epoch 459/500, Batch  

In [ ]:
!ls '/content/drive/My Drive/CVTermProject/train/data/dataset/tensors/' | wc -l

1979
